# Reinforcement learning in the gridworld

Suppose that an agent is situated in the 4×3 environment shown in the figure below. Beginning
in the start state, it must choose an action at each time step. The interaction with the environment
terminates when the agent reaches one of the goal states, marked +1 or –1. Aavailable actions are Up, Down, Left, and Right. We assume, this gridworld is deterministic, meaning  the agent will go where it intends to go. For example, when the agent decides to take action up at (0, 1), it will land in (0, 2) rather than elsewhere. 

We apply reinforcement learning to find best traveling path for the agent. 

<img src='gridworld.png' height="40%" width="40%">

We can encode the world as below:
<img src='gridworld_coded.png'  height="40%" width="42%">

In [18]:
import random

In [1]:
ALL_POSSIBLE_ACTIONS = ['U', 'D', 'L', 'R']
GOAL = (3, 2)
PIT = (3, 1)
WALL = (1, 1)

In [2]:
GOAL_REWARD = 1
PIT_REWARD = -1
WALL_REWARD = 0
STANDARD_REWARD = -0.04

In [3]:
all_the_states = [(0, 0), (0, 1), (0, 2),\
                  (1, 0), (1, 1), (1, 2),\
                  (2, 0), (2, 1), (2, 2),\
                  (3, 0), (3, 1), (3, 2)]

In [4]:
value_dictionary = {(0, 0): 0, (0, 1): 0, (0, 2): 0,\
                    (1, 0): 0, (1, 1): 0, (1, 2): 0,\
                    (2, 0): 0, (2, 1): 0, (2, 2): 0,\
                    (3, 0): 0, (3, 1): 0, (3, 2): 0}

In [7]:
def compute_reward(state):
    if state == GOAL:
        return GOAL_REWARD
    elif state == PIT:
        return PIT_REWARD
    elif state == WALL:
        return WALL_REWARD
    else:
        return STANDARD_REWARD

In [9]:
states_to_explore = [state for state in all_the_states\
                     if state != GOAL and state != PIT and state != WALL]

In [10]:
def one_step_look_ahead(current_state, action, value_dictionary):
    # output: action, state_prime, reward, value
    x = current_state[0]
    y = current_state[1]
    

    if action == 'U':
        next_state = x, y + 1
    elif action == 'R':
        next_state = x + 1, y
    elif action == 'D':
        next_state = x, y - 1
    else:
        next_state = x - 1, y

    (i, j) = next_state
    if (i, j) != (1, 1) and ((0 <= i <= 3) and (0 <= j <= 2)):
        return action, (i, j), compute_reward((i, j)), value_dictionary[(i, j)] 
    else:
        return action, None, compute_reward((i, j)), 0    

In [11]:
def max_value_state(current_state, value_dictionary):
    transitional_list = []
    for action in ALL_POSSIBLE_ACTIONS:
        transitional_list.append(one_step_look_ahead(current_state, action, value_dictionary)) 

    value_next_state_dict = {}
    for x in transitional_list:
        value_next_state_dict[x] = x[2] + GAMMA*x[3]
    
    max_value = max(value_next_state_dict.values()) 

    max_value_states = [k for k,v in value_next_state_dict.items() if v == max_value]
    
    if len(max_value_states) > 1:
        max_value_state = random.choice(max_value_states)
    else:
        max_value_state = max_value_states[0]

    return max_value_state, value_next_state_dict[max_value_state]
                

In [12]:
def value_iteration(states_to_explore, value_dictionary):
    
    while True:
        delta = 0
        for state in states_to_explore:
            old_value = value_dictionary[state]
            _, new_value = max_value_state(state, value_dictionary)
            value_dictionary[state] = new_value
            delta = max(delta, abs(new_value - old_value))
        if delta < THRESHOLD:
            break
    return value_dictionary

In [13]:
def display_value(value_dictionary):
    #print('\n')
    print("      0      1      2      3  ")
    print("  +------+------+------+------+")
    print("2 | %.2f | %.2f | %.2f | %.2f |" %(value_dictionary[(0, 2)], value_dictionary[(1, 2)], value_dictionary[(2, 2)], value_dictionary[(3, 2)]))
    print("  +------+------+------+------+")
    print("1 | %.2f | %.2f | %.2f | %.2f |" %(value_dictionary[(0, 1)], value_dictionary[(1, 1)], value_dictionary[(2, 1)], value_dictionary[(3, 1)]))
    print("  +------+------+------+------+")
    print("0 | %.2f | %.2f | %.2f | %.2f |" %(value_dictionary[(0, 0)], value_dictionary[(1, 0)], value_dictionary[(2, 0)], value_dictionary[(3, 0)]))
    print("  +------+------+------+------+")
    print("")

In [14]:
def policy_extraction(states_to_explore, value_dictionary):
    policy_dictionary = {}
    for state in states_to_explore:
        policy_dictionary[state] = max_value_state(state, value_dictionary)[0][0]
    return policy_dictionary

In [15]:
def display_policy(policy_dictionary):
    #print('G: poal, P: pit, W: wall')
    print("     0     1     2     3  ")
    print("  +-----+-----+-----+-----+")
    print("2 |  %s  |  %s  |  %s  |     |" %(policy_dictionary[(0, 2)], policy_dictionary[(1, 2)], policy_dictionary[(2, 2)]))
    print("  +-----+-----+-----+-----+")
    print("1 |  %s  |     |  %s  |     |" %(policy_dictionary[(0, 1)], policy_dictionary[(2, 1)]))
    print("  +-----+-----+-----+-----+")
    print("0 |  %s  |  %s  |  %s  |  %s  |" %(policy_dictionary[(0, 0)], policy_dictionary[(1, 0)], policy_dictionary[(2, 0)], policy_dictionary[(3, 0)]))
    print("  +-----+-----+-----+-----+")
    #print('\nG: poal, P: pit, W: wall')
    print("")

In [16]:
GAMMA = 0.9
THRESHOLD = 0.0001

## Value iteration

In [23]:
value_iterate = value_iteration(states_to_explore, value_dictionary)
display_value(value_iterate)

      0      1      2      3  
  +------+------+------+------+
2 | 0.73 | 0.86 | 1.00 | 0.00 |
  +------+------+------+------+
1 | 0.62 | 0.00 | 0.86 | 0.00 |
  +------+------+------+------+
0 | 0.52 | 0.62 | 0.73 | 0.62 |
  +------+------+------+------+



## Policy extraction

In [21]:
display_policy(policy_extraction(states_to_explore, value_dictionary))

     0     1     2     3  
  +-----+-----+-----+-----+
2 |  R  |  R  |  R  |     |
  +-----+-----+-----+-----+
1 |  U  |     |  U  |     |
  +-----+-----+-----+-----+
0 |  U  |  R  |  U  |  L  |
  +-----+-----+-----+-----+



## References

https://www.youtube.com/watch?v=14BfO5lMiuk&list=PLWzQK00nc192L7UMJyTmLXaHa3KcO0wBT&index=1

https://github.com/colinskow/move37/tree/master/dynamic_programming

https://towardsdatascience.com/reinforcement-learning-implement-grid-world-from-scratch-c5963765ebff